## Entrainment Movie Generator
- creates a movie of flashing lights and allows users to set the desired hz, the duration of the clip, etc.

In [1]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt 
import numpy as np
import time

In [2]:
desired_hz = [5,7,9] # can be a list of length 1 if you only want one frequency
# optional param if you were viewing the data in jupyter notebook: interval = 1000./desired_hz
clip_duration = 60 # seconds
frame_rate = 60 # higher frame rates improve the approximation of the frequency but most laptops have 60Hz refresh rate
frame_count = clip_duration * frame_rate # important for the duration of the video
# macbook pro 2016 retina pixel values
height = 1800 / 120 # smaller arrays will write MUCH faster
width = 2880 / 120 

In [3]:
def init():
    """initialization function for first frame"""
    im.set_data(np.zeros((height,width)))
    return im,    

In [4]:
def update_img(frame_n):
    """take as input a new frame and set that that new frame to the AxesImage object"""
    im.set_data(frame_n)
    return im,

In [5]:
def approximate_hz(desired_hz=desired_hz, frame_rate=frame_rate):
    """finds the closest frequency that can be approximated given frame_rate (FPS)"""
    # need an approximation due to the frame rate not being divisible by the frequency
    closest_approx = []
    for hz in desired_hz:        
        temp = []
        for i in range(1,frame_rate+1):
            temp.append(abs(1/float(hz) - (i*1)/float(frame_rate)))
        closest_approx.append((temp.index(min(temp))+1) / 2) # "/2" was important to ensure the switches occur quickly (e.g. 2x)
    
    return closest_approx

In [6]:
def hz_switch_point(frame_count=frame_count, desired_hz=desired_hz):
    """creates equal length segments for each frequency in desired_hz"""
    hz_switch_point_frame = frame_count / len(desired_hz) # will be used in the modulus
    
    return hz_switch_point_frame

In [7]:
def frame_gen(frame_count=frame_count, frame_rate=frame_rate, desired_hz=desired_hz):
    """generates a new frame every time it is called, keeping track of where the video is across different
    frequencies and when to switch from black to white"""
    closest_approx = approximate_hz(desired_hz=desired_hz, frame_rate=frame_rate)
    hz_switch_point_frame = hz_switch_point(frame_count=frame_count, desired_hz=desired_hz)
    
    current_frequency = -1 # zeroeth index of desired_hz (gets incremented by one in modulus in first run)
    on = True
    i = 0
    while True:
        if i % hz_switch_point_frame == 0:
            current_frequency +=1 # incremented by one in the first run
        # black
        if i % (closest_approx[current_frequency]) == 0:
            on = not on
            np_array = np.ones((height, width),dtype=int) * on # switch from black to white
            i += 1
            yield np_array
        # white
        else:
            i += 1
            yield np_array

In [8]:
fig,ax = plt.subplots()
fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
ax.axis('off')
im = ax.imshow(np.ones((height,width)),cmap='gray',vmin=0,vmax=1, interpolation='nearest',aspect="auto")

In [9]:
start = time.time()
ani = animation.FuncAnimation(fig=fig, func=update_img, frames=frame_gen, init_func=init, blit=True, \
                              save_count=frame_count) # save count is the number of times to call the frame_gen
writer = animation.writers['ffmpeg'](fps=frame_rate)

ani.save('../data/5_7_9hz_full.mp4',writer=writer,dpi=100)

print time.time() - start

42.2335650921
